Hi,

This is a draft code that loads the pretrained model and predicts a given input. 
The notebook should run in less than five minutes.

Keep in mind that you should download the following folder (my saved model) and put it in the same folder as your code.

https://mega.nz/folder/MmB1gIIT#8ilUTK1-BO80aoXxKOIhpg

-Moradnejad

##Set up environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install tensorflow-gpu

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

# Load the model

In [ ]:
import keras

model = keras.models.load_model("/content/drive/Shareddrives/NLP Final Project/Vijay Colab/ColBERT/colbert-trained/")
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_21 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_22 (InputLayer)          [(None, 20)]         0           []                               
                                                                                       

# complete code

In [ ]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_21 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_22 (InputLayer)          [(None, 20)]         0           []                               
                                                                                       

In [ ]:
import subprocess
from ast import literal_eval

def run(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    out, err = process.communicate()
    print(out.decode('utf-8').strip())

print('# CPU')
run('cat /proc/cpuinfo | egrep -m 1 "^model name"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu MHz"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu cores"')

print('# RAM')
run('cat /proc/meminfo | egrep "^MemTotal"')

print('# GPU')
run('lspci | grep VGA')

print('# OS')
run('uname -a')

# CPU
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
cpu MHz		: 2199.998
cpu cores	: 1
# RAM
MemTotal:       13302920 kB
# GPU

# OS
Linux 8e7c413e8867 5.4.144+ #1 SMP Tue Dec 7 09:58:10 PST 2021 x86_64 x86_64 x86_64 GNU/Linux


In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# import tensorflow_hub as hub
import tensorflow as tf
# import bert_tokenization as tokenization
import tensorflow.keras.backend as K
from tensorflow import keras 

import os
from scipy.stats import spearmanr
from math import floor, ceil
# from transformers import *

import seaborn as sns
import string
import re    #for regex

np.set_printoptions(suppress=True)
print(tf.__version__)

2.8.0


# Prep / tokenizer

#### 1. Read data and tokenizer

Read tokenizer and data, as well as defining the maximum sequence length that will be used for the input to Bert (maximum is usually 512 tokens)

In [ ]:
training_sample_count = 10000 #500 # 4000 # CHANGE THIS VALUE TO CHANGE TOTAL NUMBER OF SAMPLES
test_count = 4000

MAX_SENTENCE_LENGTH = 20
MAX_SENTENCES = 5
MAX_LENGTH = 100

In [ ]:
!ls /kaggle/input/200k-short-texts-for-humor-detection

ls: cannot access '/kaggle/input/200k-short-texts-for-humor-detection': No such file or directory


### original dataset

In [ ]:
# df = pd.read_csv('/kaggle/input/200k-short-texts-for-humor-detection/dataset.csv')

# df_train = pd.read_csv('/kaggle/input/200k-short-texts-for-humor-detection/train.csv')
# data = pd.read_csv('/content/drive/Shareddrives/NLP Final Project/Vijay Colab/data/task1/task1_train.csv')
# print("data.shape:", data.shape)
# print(data.label.value_counts())

# print(data.columns)
# data.drop(columns=['id'], inplace=True)
# display(data.head(3))
# print(data)
# data = data[:training_sample_count]
# print(data.shape)

# df_train, df_test = sklearn.model_selection.train_test_split(data, test_size=0.2, random_state=45)
# temp = df_test[df_test['label'] == 1][:616]
# temp1 = df_test[df_test['label'] == 0]
# df_test = pd.concat([temp, temp1]).sample(frac=1)
# # df_test.drop(columns=['id'], inplace=True)
# # display(df_test.head(3))
# # df_test = df_test[:test_count]

# # df_test = pd.read_csv('/kaggle/input/200k-short-texts-for-humor-detection/dev.csv')
# # df_test = pd.read_csv('/content/drive/Shareddrives/NLP Final Project/Vijay Colab/data/task1/task1_test.csv')
# # df_test.drop(columns=['id'], inplace=True)
# # display(df_test.head(3))
# # df_test = df_test[:test_count]

# print(df_train.label.value_counts()) # df_train.shape[0] is the number of training samples
# print(df_test.label.value_counts())

df_train = pd.read_csv('/content/drive/Shared drives/NLP Final Project/Dataset/balanced_train_task1.csv')
df_test = pd.read_csv('/content/drive/Shared drives/NLP Final Project/Dataset/balanced_test_task1.csv')
df_test.drop(columns=['id', 'Unnamed: 0'], inplace=True)
df_train.drop(columns=['id', 'Unnamed: 0'], inplace=True)

In [ ]:
test_df_y = df_test.copy()
del df_test['label']

df_sub = test_df_y.copy()

# print(len(df),len(df_train),len(df_test))
print(len(df_train),len(df_test))
display(df_train.head())
display(df_test.head())

8000 1600


,joke,label
0,当他们全都被困在一起时，他们为什么称他们为“公寓”？,0
1,总有一个白痴比你指望的还要多。,0
2,我们的父母和祖父母是一个更大的威胁！,0
3,通过用细条培根覆盖任何切口和放牧，可以对猪进行轻微的皮肤移植。,0
4,有时我们不会想你。和它一起生活。,0


,joke
0,新法规意味着新监管机构的新工作岗位。
1,我有足够的钱让我度过余生，...除非我买东西。
2,昨天今天是明天，明天今天是昨天。
3,服务员！一位顾客喊道，广告中说你们自己制作混合咖啡，但是这根本不是混合咖啡的味。 服务员回答...
4,我认为你的问题是自尊心低。在输家中很常见。


In [ ]:
print(list(df_train.columns))

['joke', 'label']


In [ ]:
output_categories = list(df_train.columns[[1]])
input_categories = list(df_train.columns[[0]])

TARGET_COUNT = len(output_categories)

print('\ninput categories:\n\t', input_categories)
print('\noutput TARGET_COUNT:\n\t', TARGET_COUNT)
print('\noutput categories:\n\t', output_categories)


input categories:
	 ['joke']

output TARGET_COUNT:
	 1

output categories:
	 ['label']


## 2. Preprocessing functions

These are some functions that will be used to preprocess the raw text data into useable Bert inputs.<br>


In [ ]:
from transformers import BertTokenizer

# MODEL_TYPE = 'bert-base-uncased'
MODEL_TYPE = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE)

In [ ]:
import nltk
nltk.download('punkt', force=True)
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def chinese_sent_tokenize(text):
  # pattern = r',|\.|/|;|\'|`|\[|\]|<|>|\?|\？|:|"|\{|\}|\~|!|@|#|\$|%|\^|&|\(|\)|-|=|\_|\+|，|。|、|；|‘|’|【|】|·|！| |…|（|）'
  pattern = r'\.|\?|\？|!|。|！'
  return list(filter(None, re.split(pattern, text)))

In [ ]:
print(chinese_sent_tokenize("你好。你喜欢苹果？我，你；你：我的名字！他说'你好'。"))
print(chinese_sent_tokenize("我喜欢结婚。很高兴找到一个你想要在余生中惹恼的特殊人物。"))

print(chinese_sent_tokenize(df_train.iloc[5].joke))

['你好', '你喜欢苹果', '我，你；你：我的名字', "他说'你好'"]
['我喜欢结婚', '很高兴找到一个你想要在余生中惹恼的特殊人物']
['儿子4岁时，大爷家的小哥哥刚上一年级', '一次，小哥哥高兴地告诉大家：“期末考试我考了第九名', '” 儿子不服气地说：“才考第九啊，我上学非考第十不可', '”']


In [ ]:
def return_id(str1, str2, truncation_strategy, length):

    inputs = tokenizer.encode_plus(str1, str2,
        add_special_tokens=True,
        max_length=length,
        truncation_strategy=truncation_strategy)

    input_ids =  inputs["input_ids"]
    input_masks = [1] * len(input_ids)
    input_segments = inputs["token_type_ids"]
    padding_length = length - len(input_ids)
    padding_id = tokenizer.pad_token_id
    input_ids = input_ids + ([padding_id] * padding_length)
    input_masks = input_masks + ([0] * padding_length)
    input_segments = input_segments + ([0] * padding_length)

    return [input_ids, input_masks, input_segments]


def compute_input_arrays(df, columns, tokenizer):
    model_input = [] # list to return
    for xx in range((MAX_SENTENCES*3)+3):
        model_input.append([])
    
    for _, row in tqdm(df[columns].iterrows()):
        i = 0
        
        # sent
        sentences = chinese_sent_tokenize(row.joke)
        for xx in range(MAX_SENTENCES):
            s = sentences[xx] if xx<len(sentences) else ''
            ids_q, masks_q, segments_q = return_id(s, None, 'longest_first', MAX_SENTENCE_LENGTH)
            model_input[i].append(ids_q)
            i+=1
            model_input[i].append(masks_q)
            i+=1
            model_input[i].append(segments_q)
            i+=1
        
        # full row
        ids_q, masks_q, segments_q = return_id(row.joke, None, 'longest_first', MAX_LENGTH)
        model_input[i].append(ids_q)
        i+=1
        model_input[i].append(masks_q)
        i+=1
        model_input[i].append(segments_q)
        
    for xx in range((MAX_SENTENCES*3)+3):
        model_input[xx] = np.asarray(model_input[xx], dtype=np.int32)
        
    print(model_input[0].shape)
    return model_input



In [ ]:
inputs      = compute_input_arrays(df_train, input_categories, tokenizer)
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer)

0it [00:00, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


(8000, 20)


0it [00:00, ?it/s]

(1600, 20)


In [ ]:
inputs[0].shape


(8000, 20)

In [ ]:
print(len(inputs), len(inputs[0]), len(inputs[0][0]))

# check out input for 7th row
xx = 7
print(df_train.iloc[xx,0])
print(chinese_sent_tokenize(df_train.iloc[xx,0]))
inputs[0][xx], inputs[3][xx], inputs[6][xx], inputs[15][xx]

18 8000 20
退后！你像廉价的内衣一样爬行。
['退后', '你像廉价的内衣一样爬行']


(array([ 101, 6842, 1400,  102,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32),
 array([ 101,  872, 1008, 2442,  817, 4638, 1079, 6132,  671, 3416, 4260,
        6121,  102,    0,    0,    0,    0,    0,    0,    0], dtype=int32),
 array([101, 102,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0], dtype=int32),
 array([ 101, 6842, 1400, 8013,  872, 1008, 2442,  817, 4638, 1079, 6132,
         671, 3416, 4260, 6121,  511,  102,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,

In [ ]:
print(type(inputs))
print(len(inputs))
print(type(inputs[0]))
print(inputs[0].shape)
print(inputs[1].shape)
print(inputs[2].shape)


<class 'list'>
18
<class 'numpy.ndarray'>
(8000, 20)
(8000, 20)
(8000, 20)


In [ ]:
def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

outputs = compute_output_arrays(df_train, output_categories)
outputs[:3]

array([[0],
       [0],
       [0]])

## 5. Training, validation and testing

Loops over the folds in gkf and trains each fold for 3 epochs --- with a learning rate of 3e-5 and batch_size of 6. A simple binary crossentropy is used as the objective-/loss-function. 

In [ ]:
type(inputs)
inputs[0]

array([[ 101, 2496,  800, ...,  720, 4917,  102],
       [ 101, 2600, 3300, ...,    0,    0,    0],
       [ 101, 2769,  812, ..., 5516,  102,    0],
       ...,
       [ 101, 6568, 1218, ..., 2769,  812,  102],
       [ 101, 1963, 3362, ..., 3398,  833,  102],
       [ 101, 5439, 1062, ..., 1408,  102,    0]], dtype=int32)

In [ ]:
model.fit(inputs, outputs, batch_size=6, epochs=7)

Epoch 1/7


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


1334/1334 [==============================] - 612s 443ms/step - loss: 0.5273
Epoch 2/7
1334/1334 [==============================] - 584s 438ms/step - loss: 0.4528
Epoch 3/7
1334/1334 [==============================] - 583s 437ms/step - loss: 0.3864
Epoch 4/7
1334/1334 [==============================] - 581s 436ms/step - loss: 0.2964
Epoch 5/7
1334/1334 [==============================] - 580s 435ms/step - loss: 0.1858
Epoch 6/7
1334/1334 [==============================] - 579s 434ms/step - loss: 0.1146
Epoch 7/7
1334/1334 [==============================] - 579s 434ms/step - loss: 0.0730


In [ ]:
# Evaluation Metrics
import sklearn
def print_evaluation_metrics(y_true, y_pred, label='', is_regression=True, label2=''):
    print('==================', label2)
    ### For regression
    if is_regression:
        print('mean_absolute_error',label,':', sklearn.metrics.mean_absolute_error(y_true, y_pred))
        print('mean_squared_error',label,':', sklearn.metrics.mean_squared_error(y_true, y_pred))
        print('r2 score',label,':', sklearn.metrics.r2_score(y_true, y_pred))
        #     print('max_error',label,':', sklearn.metrics.max_error(y_true, y_pred))
        return sklearn.metrics.mean_squared_error(y_true, y_pred)
    else:
        ### FOR Classification
#         print('balanced_accuracy_score',label,':', sklearn.metrics.balanced_accuracy_score(y_true, y_pred))
#         print('average_precision_score',label,':', sklearn.metrics.average_precision_score(y_true, y_pred))
#         print('balanced_accuracy_score',label,':', sklearn.metrics.balanced_accuracy_score(y_true, y_pred))
#         print('accuracy_score',label,':', sklearn.metrics.accuracy_score(y_true, y_pred))
        print('f1_score',label,':', sklearn.metrics.f1_score(y_true, y_pred))
        
        matrix = sklearn.metrics.confusion_matrix(y_true, y_pred)
        print(matrix)
        TP,TN,FP,FN = matrix[1][1],matrix[0][0],matrix[0][1],matrix[1][0]
        Accuracy = (TP+TN)/(TP+FP+FN+TN)
        Precision = TP/(TP+FP)
        Recall = TP/(TP+FN)
        F1 = 2*(Recall * Precision) / (Recall + Precision)
        print('Acc', Accuracy, 'Prec', Precision, 'Rec', Recall, 'F1',F1)
        return sklearn.metrics.accuracy_score(y_true, y_pred)

print_evaluation_metrics([1,0], [0.9,0.1], '', True)
print_evaluation_metrics([1,0], [1,1], '', False)

mean_absolute_error  : 0.09999999999999999
mean_squared_error  : 0.009999999999999998
r2 score  : 0.96
f1_score  : 0.6666666666666666
[[0 1]
 [0 1]]
Acc 0.5 Prec 0.5 Rec 1.0 F1 0.6666666666666666


0.5

### Loss function selection
Regression problem between 0 and 1, so binary_crossentropy and mean_absolute_error seem good.

Here are the explanations: https://www.dlology.com/blog/how-to-choose-last-layer-activation-and-loss-function/

In [ ]:
valid_inputs = inputs
valid_outputs = outputs

In [ ]:
preds = model.predict(valid_inputs)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


In [ ]:
len(valid_inputs[0])

8000

In [ ]:
print(valid_outputs.shape, preds.shape)
print_evaluation_metrics(np.array(valid_outputs), np.array(preds), '')

(8000, 1) (8000, 1)
mean_absolute_error  : 0.011086497086927011
mean_squared_error  : 0.005034878142286989
r2 score  : 0.9798604874308521


0.005034878142286989

In [ ]:
test_preds = model.predict(test_inputs)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


In [ ]:
import torch

torch.save(model, "drive/Shared drives/NLP Final Project/colbert")

INFO:tensorflow:Assets written to: ram://eb942339-3e81-41bf-a9bd-b833abd7e1ed/assets


INFO:tensorflow:Assets written to: ram://eb942339-3e81-41bf-a9bd-b833abd7e1ed/assets


## Binary submission

In [ ]:
for split in np.arange(0.1, 0.99, 0.1).tolist():
    df_sub['pred_bi'] = (test_preds > split)

    print_evaluation_metrics(df_sub['label'], df_sub['pred_bi'], '', False, 'SPLIT on '+str(split))

    df_sub.to_csv('sub3.csv', index=False)
    df_sub.head()

================== SPLIT on 0.1
f1_score  : 0.7415730337078651
[[648 152]
 [239 561]]
Acc 0.755625 Prec 0.7868162692847125 Rec 0.70125 F1 0.7415730337078651
================== SPLIT on 0.2
f1_score  : 0.7424657534246576
[[682 118]
 [258 542]]
Acc 0.765 Prec 0.8212121212121212 Rec 0.6775 F1 0.7424657534246576
================== SPLIT on 0.30000000000000004
f1_score  : 0.7447698744769874
[[700 100]
 [266 534]]
Acc 0.77125 Prec 0.8422712933753943 Rec 0.6675 F1 0.7447698744769874
================== SPLIT on 0.4
f1_score  : 0.7459535538353272
[[709  91]
 [270 530]]
Acc 0.774375 Prec 0.8534621578099839 Rec 0.6625 F1 0.7459535538353272
================== SPLIT on 0.5
f1_score  : 0.7437544610992148
[[720  80]
 [279 521]]
Acc 0.775625 Prec 0.8668885191347754 Rec 0.65125 F1 0.7437544610992148
================== SPLIT on 0.6
f1_score  : 0.7449567723342939
[[729  71]
 [283 517]]
Acc 0.77875 Prec 0.8792517006802721 Rec 0.64625 F1 0.7449567723342939
================== SPLIT on 0.7000000000000001
f1_

In [ ]:
print(np.sum(df_sub['label'] == 0))
print(np.sum(df_sub['label'] == 1))
print(df_sub.shape)
print(df_train.shape)
print(df_test.shape)
df_train.head()

800
800
(1600, 3)
(8000, 2)
(1600, 1)


,joke,label
0,当他们全都被困在一起时，他们为什么称他们为“公寓”？,0
1,总有一个白痴比你指望的还要多。,0
2,我们的父母和祖父母是一个更大的威胁！,0
3,通过用细条培根覆盖任何切口和放牧，可以对猪进行轻微的皮肤移植。,0
4,有时我们不会想你。和它一起生活。,0


In [ ]:
df_sub['pred_bi'] = (test_preds > 0.5)

print_evaluation_metrics(df_sub['humor'], df_sub['pred_bi'], '', False, 'SPLIT on '+str(split))

df_sub.to_csv('sub.csv', index=False)
df_sub.head()

KeyError: ignored

In [ ]:
print('Texts that the model failed to correctly predict:')
df_sub[df_sub['pred_bi']!=df_sub['humor']]

In [ ]:
import torch

torch.save(model, "colbert")